# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096935


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:37,  3.47s/it]

Rendering models:  10%|█         | 3/30 [00:08<01:19,  2.93s/it]

Rendering models:  13%|█▎        | 4/30 [00:13<01:32,  3.54s/it]

Rendering models:  17%|█▋        | 5/30 [00:15<01:13,  2.93s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:48<00:04,  1.52s/it]

Rendering models:  93%|█████████▎| 28/30 [00:50<00:03,  1.66s/it]

Rendering models:  97%|█████████▋| 29/30 [00:53<00:02,  2.03s/it]

Rendering models: 100%|██████████| 30/30 [00:55<00:00,  1.85s/it]

not-logged-in-3b0b929f4475fd276561    0.000221
not-logged-in-3b0b929f4475fd276561    0.000189
not-logged-in-3b0b929f4475fd276561    0.000179
not-logged-in-3b0b929f4475fd276561    0.000184
Jnursssmith5263                       0.000230
aliabdulhameed                        0.000182
not-logged-in-808d8d7eb7071532ff8d    0.019256
jnarayanbvg                           0.001056
earthisflat                           0.039204
not-logged-in-a3d7379de3dae98540ca    0.000207
froggler                              0.000182
kc_                                   0.000381
acapirala                             0.000212
Susymuller                            0.000187
rhondacard                            0.000253
Lavadude                              0.000184
not-logged-in-6ea7dd94a335e5238780    0.000258
BigTribble                            0.000205
not-logged-in-4a99265f22da29adddc5    0.000192
r.sundstrom3                          0.000244
Cinimod                               0.000181
bboyle23     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())